# Compare two ROI files and return the score and the ROIs of False Positives and False Negatives detection

Compare two ROI .zip files, one containing manual annotation of cell event, and the one containing the DeXtrusion detection, and print the accuracy scores. It can also generate ROI .zip file containing the False Positives and False Negatives (of DeXtrusion results compared to manual one) files to allow to check them.

In [35]:
# default parameters
talkative = True  ## print info messages
import os
from glob import glob
import numpy as np
imname = os.getcwd()
# DeXNet path, to read the configuration file
modeldir = "./deXNets/notum_all/notum_all_original/notumAll1"
from dextrusion.DeXtrusion import DeXtrusion
dexter = DeXtrusion(verbose=talkative)

Tensorflow with Cuda: False
Tensorflow version: 2.15.0
Num GPUs Available:  0


In [52]:
#### Parameters to choose

dxy = 15          ## distance to ground-truth ROI to consider same
dt = 4            ## distance in time to ground-truth ROI to consider same
cat = 1           ## event to score (here cell_death)
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]

## path the ROIs files are
test_path = "../../data/notum_retrain_mix/test/"
## path the results folder where DeXtrusion ROI files are
res = test_path+"results/"

# parameters used to create the ROIs (to keep in memory in the score datasheet)
volume_thres = 800
prob_thres = 180

In [53]:
## parameter to consider ROIs as the same within DeXtrusion (spatial and temporal distance threshold)
distxy=10
dist=4

### Write results to file
resultfile = res+"/score.csv"
header = 'Name;VolumeThreshold;ProbaThreshold;distanceXY;distanceT;TP;FP;FN;Precision;Recall' 

## Test all the files present in the result folder
resfiles = glob(res+"/*"+catnames[cat], recursive = False)
resarray = None

for resfile in resfiles:
    testname = os.path.basename(resfile)
    ind = testname.find(catnames[cat])
    testname = testname[0:ind]
    print(testname)
    paras = np.array([testname, volume_thres, prob_thres, dxy, dt])
    outfile = res+testname+catnames[cat]
    #dexter.get_rois_of_scaled_image( res+testname+catnames[cat]+"_proba.tif", volume_thres, prob_thres, distxy, dist, cat=cat, outfile=outfile)
    
    # compare the ROIs and print the scores
    score = dexter.compare_rois(cat=cat, gtroisfile=test_path+testname+catnames[cat], resroisfile=outfile, distance_xy=dxy, distance_t=dt)
    
    # write in .zip files the false detections (put # in the beginning of the lines to not do this step)
    dexter.write_falsepositives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat], distance_xy=dxy, distance_t=dt)
    dexter.write_falsenegatives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat],distance_xy=dxy, distance_t=dt)
    
    array = np.atleast_2d(np.append(paras, np.array(score)))
    if resarray is None:
        resarray = array
    else:
        resarray = np.append(resarray, array, axis=0)

## save the scores in a tabular file
with open(resultfile,'w') as csvfile:
    np.savetxt(csvfile, resarray, delimiter=';', header=header, fmt='%s', comments='')

000_029
True positives 4
False positives 2
False negatives 3
Precision 0.6666666666666666
Recall 0.5714285714285714
200_229
True positives 4
False positives 1
False negatives 4
Precision 0.8
Recall 0.5


Mix two proba_tif

In [ ]:
import os, glob
import numpy as np
import tifffile as tiff

# ----- Config -----
cat = 1
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]

test_path = "/Users/yuyangsmacbook/project/data"
res1      = os.path.join(test_path, "notum0_retrain/test/results")
res2      = os.path.join(test_path, "notum1_retrain/test/results")
res_ens   = os.path.join(test_path, "notum_retrain_mix/test/results")
os.makedirs(res_ens, exist_ok=True)

# Important: drop ".zip" to match files like *_cell_death_proba.tif
pattern = "*" + catnames[cat].replace(".zip","") + "_proba.tif"

def collect(d):
    m = {}
    for p in glob.glob(os.path.join(d, pattern)):
        base = os.path.basename(p).replace("_proba.tif","")  # e.g. A_cell_death
        m[base] = p
    return m

m1 = collect(res1)
m2 = collect(res2)
keys = sorted(set(m1) & set(m2))

print(f"Ensembling {len(keys)} samples")
if not keys:
    print("No matching proba maps found. Check paths and naming.")
    print("res1:", res1)
    print("res2:", res2)

for base in keys:
    p1, p2 = m1[base], m2[base]
    im1, im2 = tiff.imread(p1), tiff.imread(p2)

    if im1.shape != im2.shape:
        print("Skip (shape mismatch):", base, im1.shape, im2.shape)
        continue

    # Average in float, then cast back to original dtype
    avg = (im1.astype(np.float32) + im2.astype(np.float32)) / 2.0

    if np.issubdtype(im1.dtype, np.integer):
        info = np.iinfo(im1.dtype)
        avg = np.clip(avg, info.min, info.max).astype(im1.dtype)
    else:
        # Float case (e.g., 0..1 probability): keep float dtype
        avg = avg.astype(im1.dtype)

    outp = os.path.join(res_ens, base + "_proba.tif")  # e.g. A_cell_death_proba.tif
    tiff.imwrite(outp, avg, imagej=True)
    print("Wrote:", outp)

print("Done. Outputs in:", res_ens)


将融合 2 个样本
wrote: /Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results/000_029_cell_death_proba.tif
wrote: /Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results/200_229_cell_death_proba.tif
